# Linear regression을 활용한 부동산 예측

## 1.Import Required Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom
viz = Visdom()


## 2. Data Loader

In [2]:
num_data=1000
num_epoch=200

import pandas as pd # 엑셀 데이터 처리를 위한 pandas패키지 로드

df = pd.read_csv("price.csv", parse_dates =["date"], index_col ="date")  #엑셀 파일을 읽어 df변수에 저장
df = df.dropna()
#df=(df-df.mean())/df.std()
#df=(df-df.min())/(df.max()-df.min())+0.0001

df.head()


,tradeprice_sido_n1,region_cd,tradeprice_sido,year,month,building_type,construction_realized_amount,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,,,,,,,
2010-01-01,93.4,45000,93.3,2010,1,7,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,0.0,0.0
2010-01-01,83.9,30000,83.1,2010,1,0,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0
2010-01-01,92.7,30000,92.3,2010,1,7,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0
2010-01-01,80.8,30000,79.7,2010,1,1,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0
2010-01-01,81.8,30000,81.8,2010,1,3,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0


In [3]:
df.resample(rule='A').mean() #연도별로 통계를 내보자

,tradeprice_sido_n1,region_cd,tradeprice_sido,year,month,building_type,construction_realized_amount,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,,,,,,,
2010-12-31,85.678906,36750.000000,85.469271,2010.0,6.500000,2.75,7.689668e+06,2.671667,3.181667,1159.400000,1761.925833,1.475000,3.718333,568365.566667,349610.691667,1236.640625,541.677083
2011-12-31,90.873037,36727.748691,90.302749,2011.0,6.502618,2.75,7.633607e+06,3.437016,3.686492,1107.846597,1983.284188,0.725654,3.620681,615922.894764,377893.788482,683.026178,423.408377
2012-12-31,94.025387,36742.268041,94.015464,2012.0,6.551546,2.75,7.411050e+06,3.293041,3.424485,1124.052577,1931.431649,0.526804,3.130155,645835.318557,396972.789175,454.520619,186.876289
2013-12-31,94.198652,36705.882353,94.098775,2013.0,6.500000,2.75,8.174064e+06,2.722500,2.703333,1095.433333,1961.248333,0.850000,2.793333,666946.983333,406966.191667,547.215686,239.333333
2014-12-31,95.477340,36684.729064,95.375739,2014.0,6.507389,2.75,8.207670e+06,2.490887,2.420099,1052.333990,1982.338030,0.700493,2.591281,710863.112315,436247.054187,248.014778,108.068966
2015-12-31,97.420098,36705.882353,97.220221,2015.0,6.500000,2.75,8.551165e+06,1.770000,1.719167,1133.083333,2010.792500,0.800000,1.799167,772351.316667,476190.525000,180.446078,62.382353
2016-12-31,98.678342,36653.465347,98.643317,2016.0,6.514851,2.75,1.001116e+07,1.493317,1.466188,1163.131188,1986.238416,0.626238,1.441683,859289.918317,518222.086634,341.965347,50.356436
2017-12-31,99.529774,36608.040201,99.435427,2017.0,6.512563,2.75,1.152759e+07,1.444372,1.514221,1122.084422,2312.495829,0.704523,1.805427,939928.797487,561827.164322,541.894472,93.417085
2018-12-31,100.211224,36571.428571,100.162755,2018.0,2.000000,2.75,1.073421e+07,1.653265,1.769592,1069.693878,2469.895102,1.000000,2.247347,980192.414286,580582.510204,358.591837,58.204082


In [4]:
'''
‘region_cd’: ‘지역코드(시도)’,
‘year’: ‘연도’,
‘month’:’월’,
‘building_type’: ‘부동산타입’,
‘tradeprice_sido’ : ‘매매가격지수(시도)’,
‘construction_realized_amount’ : ‘건설기성액(백만원)’,
“cd”: “cd(91일물)“,
‘spirit_deposit_rate’: ‘정기예금금리’,
‘exchange_rate’: ‘환율’,
‘composite_stock_price_index’: ‘종합주가지수’,
‘economy_growth’: ‘경제성장률’,
‘exchequer_bond_three’ : ‘국고채3년‘,
‘household_loan_all’: ‘가계대출액(전국)’,
‘mortgage_all’ : ‘주택대출액(전국)’,
‘numberofnosells’:’미분양 가구수(시도)’,
‘unsalenum_c’:’공사완료후 미분양(민간,시도)’
'''
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


#train = df[(df.year > 2006) & (df.year < 2017) & (df.region_cd == 45000)]
#test = df[df.year >= 2017 & (df.region_cd == 45000)]
train = df[(df.year > 2006) & (df.year < 2017)]
test = df[df.year >= 2017]
train = normalize(train) # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!
test = normalize(test)  # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!
'''
feature_names = ['region_cd', 'year', 'month', 'building_type',
                 'tradeprice_sido', 'construction_realized_amount','cd',
                 'spirit_deposit_rate','exchange_rate','composite_stock_price_index',
                 'economy_growth','exchequer_bond_three','household_loan_all',
                 'mortgage_all','numberofnosells','unsalenum_c']  # 학습에 사용될 항목들을 설정
'''
feature_names = [
                 'tradeprice_sido','cd',
                 'spirit_deposit_rate','exchange_rate','composite_stock_price_index',
                 'economy_growth','exchequer_bond_three','household_loan_all',
                 'mortgage_all','numberofnosells','unsalenum_c']  # 학습에 사용될 항목들을 설정

input_dim = len(feature_names) # 학습에 사용될 항목들의 갯수 == 한개의 input의 feature 갯수
X_train = train[feature_names]
X_test = test[feature_names]

lable_name = "tradeprice_sido_n1"  # 어떤 값을 예측값으로 넣을것인가 이 경우 다음달 매매가격지수를 예측하도록함
Y_train = train[lable_name]
Y_test = test[lable_name]


print(X_train.shape)
print(X_test.shape)
X_train.head()  #데이터가 어떻게 생겼나 보는 함수

(5560, 11)
(992, 11)


,tradeprice_sido,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,
2010-01-01,0.679671,0.684444,1.0,0.644274,0.150134,1.0,1.0,0.0,0.0,0.000000,0.000000
2010-01-01,0.470226,0.684444,1.0,0.644274,0.150134,1.0,1.0,0.0,0.0,0.136113,0.026827
2010-01-01,0.659138,0.684444,1.0,0.644274,0.150134,1.0,1.0,0.0,0.0,0.136113,0.026827
2010-01-01,0.400411,0.684444,1.0,0.644274,0.150134,1.0,1.0,0.0,0.0,0.136113,0.026827
2010-01-01,0.443532,0.684444,1.0,0.644274,0.150134,1.0,1.0,0.0,0.0,0.136113,0.026827


In [5]:
X_test.head()

,tradeprice_sido,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,
2017-01-01,0.896552,0.428571,0.108108,1.0,0.0,0.75,0.0,0.0,0.0,0.02283,0.012873
2017-01-01,0.689655,0.428571,0.108108,1.0,0.0,0.75,0.0,0.0,0.0,0.02283,0.012873
2017-01-01,0.264368,0.428571,0.108108,1.0,0.0,0.75,0.0,0.0,0.0,0.12108,0.000000
2017-01-01,0.758621,0.428571,0.108108,1.0,0.0,0.75,0.0,0.0,0.0,0.12108,0.000000
2017-01-01,0.540230,0.428571,0.108108,1.0,0.0,0.75,0.0,0.0,0.0,0.12108,0.000000


## 3. Model & Optimizer

In [6]:
model = nn.Sequential(
            nn.Linear(input_dim,1),
   #         nn.ReLU(),
   #         nn.Linear(200,100),
   #         nn.ReLU(),
   #         nn.Linear(100,50),
   #         nn.ReLU(),
   #         nn.Linear(50,50),
   #         nn.ReLU(),
   #         nn.Linear(50,1),
        )

loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

In [7]:
print(torch.from_numpy(X_train.values))

tensor([[0.6797, 0.6844, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.4702, 0.6844, 1.0000,  ..., 0.0000, 0.1361, 0.0268],
        [0.6591, 0.6844, 1.0000,  ..., 0.0000, 0.1361, 0.0268],
        ...,
        [0.7988, 0.0889, 0.0913,  ..., 1.0000, 0.0000, 0.0000],
        [0.7659, 0.0889, 0.0913,  ..., 1.0000, 0.0000, 0.0000],
        [0.7803, 0.0889, 0.0913,  ..., 1.0000, 0.0000, 0.0000]],
       dtype=torch.float64)


## 4. Train 

In [8]:
input_data = torch.from_numpy(X_train.values).float()
label = torch.from_numpy(Y_train.values).float()
print(input_data.size())
loss_arr =[]

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(input_data)
    #print("output", output)
    #print("label", label)
    
    loss = loss_func(output,label)
    loss.backward()
    optimizer.step()
    
    loss_arr.append(loss.item())
    print(loss.item())


torch.Size([5560, 11])
0.6415968537330627
0.5972029566764832
0.5708562731742859
0.5562342405319214
0.5487434267997742
0.5450578927993774
0.5434046387672424
0.5427204966545105
0.5427131056785583
0.5427131056785583
0.5350488424301147
0.48552578687667847
0.4412570595741272
0.40239039063453674
0.36470404267311096
0.33106574416160583
0.3058711886405945
0.28403329849243164
0.27349716424942017
0.272805392742157
0.2700422704219818
0.24593088030815125
0.22442631423473358
0.2054305076599121
0.18790726363658905
0.17500105500221252
0.16320659220218658
0.155593603849411
0.1537184864282608
0.15251073241233826
0.1494661271572113
0.14665140211582184
0.14424125850200653
0.1422586590051651
0.14066797494888306
0.13946720957756042
0.1385277360677719
0.13768181204795837
0.13688188791275024
0.13617439568042755
0.13550449907779694
0.1347583532333374
0.13403932750225067
0.13336730003356934
0.13274358212947845
0.13217265903949738
0.1316099315881729
0.1310884803533554
0.13057003915309906
0.13009113073349
0.1295

In [9]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[ 0.4144,  0.0429,  0.3565,  0.1513, -0.0585,  0.0872, -0.0700,  0.0025,
          0.2972, -0.1778,  0.1034]], requires_grad=True), Parameter containing:
tensor([0.0738], requires_grad=True)]


## 5. Visualize Trained Output

In [10]:
'''
‘region_cd’: ‘지역코드(시도)’,
‘year’: ‘연도’,
‘month’:’월’,
‘building_type’: ‘부동산타입’,
‘tradeprice_sido’ : ‘매매가격지수(시도)’,
‘construction_realized_amount’ : ‘건설기성액(백만원)’,
“cd”: “cd(91일물)“,
‘spirit_deposit_rate’: ‘정기예금금리’,
‘exchange_rate’: ‘환율’,
‘composite_stock_price_index’: ‘종합주가지수’,
‘economy_growth’: ‘경제성장률’,
‘exchequer_bond_three’ : ‘국고채3년‘,
‘household_loan_all’: ‘가계대출액(전국)’,
‘mortgage_all’ : ‘주택대출액(전국)’,
‘numberofnosells’:’미분양 가구수(시도)’,
‘unsalenum_c’:’공사완료후 미분양(민간,시도)’
'''

'\n‘region_cd’: ‘지역코드(시도)’,\n‘year’: ‘연도’,\n‘month’:’월’,\n‘building_type’: ‘부동산타입’,\n‘tradeprice_sido’ : ‘매매가격지수(시도)’,\n‘construction_realized_amount’ : ‘건설기성액(백만원)’,\n“cd”: “cd(91일물)“,\n‘spirit_deposit_rate’: ‘정기예금금리’,\n‘exchange_rate’: ‘환율’,\n‘composite_stock_price_index’: ‘종합주가지수’,\n‘economy_growth’: ‘경제성장률’,\n‘exchequer_bond_three’ : ‘국고채3년‘,\n‘household_loan_all’: ‘가계대출액(전국)’,\n‘mortgage_all’ : ‘주택대출액(전국)’,\n‘numberofnosells’:’미분양 가구수(시도)’,\n‘unsalenum_c’:’공사완료후 미분양(민간,시도)’\n'

## 6. Prediction

In [11]:
eval_data = torch.from_numpy(X_test.values).float()
eval_label = torch.from_numpy(Y_test.values).float()

eval_result = model(eval_data)
print(eval_result)
print(torch.nn.functional.sigmoid(eval_result))
print(eval_label)




tensor([[0.7163],
        [0.6306],
        [0.4355],
        [0.6403],
        [0.5498],
        [0.5451],
        [0.5246],
        [0.5007],
        [0.4636],
        [0.3778],
        [0.5064],
        [0.4493],
        [0.4281],
        [0.5411],
        [0.4888],
        [0.4888],
        [0.4697],
        [0.4662],
        [0.4618],
        [0.4475],
        [0.4571],
        [0.5380],
        [0.4672],
        [0.4910],
        [0.5101],
        [0.6382],
        [0.4524],
        [0.5858],
        [0.5906],
        [0.4386],
        [0.5809],
        [0.6285],
        [0.4720],
        [0.5404],
        [0.4975],
        [0.4023],
        [0.5261],
        [0.4767],
        [0.4720],
        [0.5055],
        [0.4817],
        [0.4475],
        [0.4809],
        [0.4475],
        [0.4380],
        [0.5053],
        [0.4428],
        [0.5857],
        [0.6142],
        [0.7286],
        [0.4285],
        [0.5523],
        [0.5220],
        [0.4982],
        [0.4077],
        [0

/Users/lim/anaconda3/envs/NLPApps/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [12]:

def normalize2(df, df_e):
    result = df_e.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df_e[feature_name] - min_value) / (max_value - min_value)
    return result


df_e = pd.read_csv("my_data.csv", parse_dates =["date"], index_col ="date")  #엑셀 파일을 읽어 df_e변수에 저장
df_e = df_e.dropna()
test_data = normalize2(df, df_e)  # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!

test_input = test_data[feature_names] # 학습에 사용했던 feature들만 뽑음
test_input.head()

test_tensor = torch.from_numpy(test_input.values).float() # pytorch tensor 형태로 데이터 변환

print(model(test_tensor))




tensor([[0.8247],
        [0.8093],
        [0.7689],
        [0.8055],
        [0.7893],
        [0.7884],
        [0.7702],
        [0.7660]], grad_fn=<AddmmBackward>)
